In [ ]:
!pip install fastapi uvicorn nest_asyncio pyngrok
!ngrok config add-authtoken 33fSFHwZWMTLsBmwzMitKb2HRqO_5vLvpC7JRQRbcejRFwnUH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import nest_asyncio
from pyngrok import ngrok
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import uvicorn
import asyncio

In [ ]:
# Allow nested event loops (required in Colab)
nest_asyncio.apply()

In [ ]:
# Create FastAPI app
app = FastAPI(title="FastAPI Chat App (Colab-Safe)")
chat_history = []
class Message(BaseModel):
    sender: str
    text: str
@app.post("/chat")
def send_message(message: Message):
    chat_history.append(message.dict())
    return {"status": "Message received", "message": message}
@app.get("/history", response_model=List[Message])
def get_history():
    return chat_history

In [ ]:
# Kill previous tunnels
!killall ngrok
ngrok.kill()

In [ ]:
# Start ngrok tunnel
port = 8000 # Changed port number
from pyngrok import ngrok
public_url = ngrok.connect(port).public_url
print(f"Public URL: {public_url}\nDocs: {public_url}/docs")

Public URL: https://unemotional-showiest-benito.ngrok-free.dev
Docs: https://unemotional-showiest-benito.ngrok-free.dev/docs


In [ ]:
from pyngrok import ngrok

try:
    tunnels = ngrok.get_tunnels()
    if tunnels:
        print("Active ngrok tunnels:")
        for tunnel in tunnels:
            print(f"- {tunnel.public_url} ({tunnel.proto}://{tunnel.config['addr']})")
    else:
        print("No active ngrok tunnels found.")
except Exception as e:
    print(f"An error occurred while checking ngrok tunnel status: {e}")

Active ngrok tunnels:
- https://unemotional-showiest-benito.ngrok-free.dev (https://http://localhost:8000)


In [ ]:
# Configure uvicorn without asyncio.run()
import uvicorn
config = uvicorn.Config(app=app, host="0.0.0.0", port=port, log_level="info")
server = uvicorn.Server(config)

In [ ]:
# Run inside the existing event loop
loop = asyncio.get_event_loop()
loop.create_task(server.serve())

<Task pending name='Task-1' coro=<Server.serve() running at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69>>